# Comparison between KS goodness-of-fit test and Bayesian model selection

We want to determine which model selection method performs better when the sample size is small. High-delay signals from real data are used for model fitting in this notebook. 

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import hera_pspec as hp
from pyuvdata import UVData
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import integrate,stats

## 1.  Estimating the power spectra for two baseline pairs (auto-baseline pspec)

In [ ]:
# Load beam model
beamfile = 'HERA_NF_dipole_power.beamfits'
cosmo = hp.conversions.Cosmo_Conversions()
uvb = hp.pspecbeam.PSpecBeamUV(beamfile, cosmo=cosmo)

In [ ]:
# Load data into UVData objects
dfile = 'zen.2458101.clean-002.uvh5'
uvd = UVData()
uvd.read(dfile)

*Run the block below for systematic model subtraction (if needed).*

In [ ]:
# mdfile = 'zen.2458101.xtmdl.uvh5' 
# #zen.2458116.34176.xx.HH.uvOCRS #zen.2458101.clean-002.uvh5 #zen.2458101.xtmdl.uvh5
# muvd = UVData()
# muvd.read(mdfile)

# bl1 = (66, 67, 'xx')
# bl2 = (83, 84, 'xx')
# bl3 = (37, 38, 'xx')

# blt_inds = uvd.antpair2ind(bl1)
# uvd.data_array[blt_inds, 0, :, 0] -= muvd.get_data(bl1)

# blt_inds = uvd.antpair2ind(bl2)
# uvd.data_array[blt_inds, 0, :, 0] -= muvd.get_data(bl2)

# blt_inds = uvd.antpair2ind(bl3)
# uvd.data_array[blt_inds, 0, :, 0] -= muvd.get_data(bl3)

In [ ]:
# find conversion factor from Jy to mK
Jy_to_mK = uvb.Jy_to_mK(np.unique(uvd.freq_array), pol='xx')
uvd.data_array *= Jy_to_mK[None, None, :, None]

In [ ]:
# We only have 1 data file here, so slide the time axis by one integration 
# to avoid noise bias (not normally needed!)
uvd1 = uvd.select(times=np.unique(uvd.time_array)[16:44:2], inplace=False)
uvd2 = uvd.select(times=np.unique(uvd.time_array)[17:45:2], inplace=False)

In [ ]:
# Create a new PSpecData object
ds = hp.PSpecData(dsets=[uvd1, uvd2], wgts=[None, None], beam=uvb)
ds.rephase_to_dset(0) # Phase to the zeroth dataset

In [ ]:
# Specify which baselines to include
baselines = [(66, 67), (83, 84), (37, 38)]

# Define uvp
uvp = ds.pspec(baselines, baselines, (0, 1), [('xx', 'xx')], spw_ranges=[(520, 690)],  
               input_data_weight='identity',
               norm='I', taper='blackman-harris', verbose=False) 

In [ ]:
# get delays
spw = 0
dlys = uvp.get_dlys(spw) * 1e9

In [ ]:
# pspec with sys. features
blp = ((66, 67), (66, 67))
key1 = (spw, blp, 'xx')
power1 = np.real(uvp.get_data(key1))

# pspec without sys. features
blp = ((83, 84), (83, 84))
key2 = (spw, blp, 'xx')
power2 = np.real(uvp.get_data(key2))

## 2. Define models ($\mathcal{N}$, $\mathcal{CNN}$ and $\Delta\mathcal{CNN}$)

### 2.1 Gaussian distribution ($N$)

In [ ]:
def gaus_pdf(x, mu, sig):
    """
    Compute the PDF of the Gaussian distribution
    
    Parameters
    ----------
    x : float, array_like
        Input data.
    mu : float
        Mean of the data.
    sig : float
        Standard deviation of the data.
    
    Return
    -------
    p : float, array_like
        Return the PDF.
    """
    
    pdf = np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
    return pdf

def gaus_cdf(x, mu, sig, range_start, range_end):
    """
    Compute the CDF of the Gaussian distribution
    
    Parameters
    ----------
    x : float, array_like
        Input data.
    mu : float
        Mean of the data.
    sig : float
        Standard deviation of the data.
    range_start, range_end : float
        Minimum and maximum value in the data.
    
    Return
    -------
    np.array(values) : array
        Return the CDF.
    
    """
    values = []
    for value in x:
        integral = integrate.quad(lambda k: gaus_pdf(k, mu, sig),range_start,value)[0]
        normalized = integral/integrate.quad(lambda k: gaus_pdf(k, mu, sig),range_start,range_end)[0]
        values.append(normalized)
    return np.array(values)

### 2.2 Complex double Gaussian distribution ($\mathcal{CNN}$)

In [ ]:
def real_pdf(z, s):
    """
    Compute the PDF of the CNN distribution
    
    Parameters
    ----------
    z : float, array_like
        Input data.
    s : float
        Standard deviation of the input data.
    
    Return
    -------
    p : float, array_like
        Return the PDF.
    """
    a = 1/(s)
    b = (-np.abs(2*z))/(s)
    p = a*np.exp(b)
    return p

def real_cdf(x, s, range_start, range_end):
    """
    Compute the CDF of the CNN distribution
    
    Parameters
    ----------
    x : float, array_like
        Input data.
    s : float
        Standard deviation of the input data.
    range_start, range_end : float
        Minimum and maximum value in the data.
    
    Return
    -------
    np.array(values) : array
        Return the CDF.
    """
    values = []
    for value in x:
        integral = integrate.quad(lambda k: real_pdf(k,s),range_start,value)[0]
        normalized = integral/integrate.quad(lambda k: real_pdf(k,s),range_start,range_end)[0]
        values.append(normalized)
    return np.array(values)

### 2.3 Distribution of the difference between two complex double Gaussian distributions ($\Delta\mathcal{CNN}$)

In [ ]:
def null_pdf(x, s):    
    """
    Compute the PDF of the delta delta CNN distribution
    
    Parameters
    ----------
    x : float, array_like
        Input data.
    s : float
        Standard deviation of the input data.
    
    Return
    -------
    pdf : float, array_like
        Return the PDF.
    """
    pdf = (1/(2*s**2))*np.exp(-2*np.abs(x)/(s))*(s+2*np.abs(x))
    return pdf

def null_cdf(x, s, range_start, range_end):
    """
    Compute the CDF of the delta CNN distribution
    
    Parameters
    ----------
    x : float, array_like
        Input data.
    s : float
        Standard deviation of the input data.
    range_start, range_end : float
        Minimum and maximum value in the data.
    
    Return
    -------
    np.array(values) : array
        Return the CDF.
    """
    values = []
    for value in x:
        integral = integrate.quad(lambda k: null_pdf(k,s),range_start,value)[0]
        normalized = integral/integrate.quad(lambda k: null_pdf(k,s),range_start,range_end)[0]
        values.append(normalized)
    return np.array(values)

## 3. Define Bayesian model selection function

In [ ]:
def bayes_factor_approx(data, m1, m2, num_bin='auto'):
    """
    Estimate twice the log of bayes factor for a set of data fitted with two models.
    
    Parameters
    ----------
    data : array_like
           Input data. The SIC is computed over the flattened array.
    m1, m2 : callable
           PDFs. For normal distribution PDF, use 'norm'.
    num_bin : integer
           Number of histogram bins. Default: 'auto'
    
    Returns
    -------
    s : float
           Return the Schwarz criterion.
    """

    # get normalized histogram data
    y, x = np.histogram(data, bins=num_bin, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0  # taking the middle value between bin edges
    
    # fit the models to the histogramed data
    popt1 = curve_fit(m1, x, y, p0=np.std(data))[0]
    if m2 == 'norm':
        popt2 = norm.fit(data)
    else:
        popt2 = curve_fit(m2, x, y, p0=np.std(data))[0]
    
    n = len(data)
        
    # compute S
    bic1 = sum(np.log(m1(data, *popt1)))
    if m2 == 'norm':
        bic2 = sum(np.log(norm.pdf(data, *popt2)))
    else:
        bic2 = sum(np.log(m2(data, *popt2)))
    p = (len(popt1)-len(popt2))/np.log(n)
    
    # estimate twice the log of Bayes factor
    s = 2*(bic1-bic2-0.5*p)

    return s

## 4. Estimating twice the log of Bayes factors

### 4.1 Fitting the $\mathcal{CNN}$ or $\mathcal{N}$ distribution to the data

We take 51 data sets of different sizes from the power spectrum with clear features of reflection systematics (power1). We change the data size by increasing the upper limit of the delay range (+60ns/group) while fixing the lower limit (~1325 ns). The data are fitted with $\mathcal{CNN}$ and $\mathcal{N}$. We estimate twice the log of Bayes factor and performed the KS goodness-of-fit tests for each fitting. The Bayesian stats and KS stats are plotted as a function of the sample size.

In [ ]:
# estimating 2log(BF)
bics = []
size = []
for i in range(51):
    x = power1[:,107:118+i].flatten()    # change sample size for each loop
    size.append(len(x))
    bics.append(bayes_factor_approx(x, real_pdf, 'norm'))

In [ ]:
# perform KS goodness-of-fit test
norm_dn = []
real_dn = []
ks_cv = []

for i in range(51):
    data = power1[:,107:118+i].flatten()
    npt = norm.fit(data)
    norm_dn.append(stats.kstest(data, lambda x: gaus_cdf(x, npt[0], npt[1], 
                                                           min(data), max(data)))[0])

    y, x = np.histogram(data, bins='auto', density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0
    popt = curve_fit(real_pdf, x, y, p0=np.std(data))[0]
    real_dn.append(stats.kstest(data, lambda x: real_cdf(x, popt, min(data), max(data)))[0])   
    
    ks_cv.append(1.36/np.sqrt(len(data)))

### 4.2 Fitting the $\Delta\mathcal{CNN}$ or $\mathcal{N}$ distribution to the data

Likewise, we have 51 data sets but with data being the differences between power spectra with and without clear systematic features and we fit the $\Delta\mathcal{CNN}$ or $\mathcal{N}$ distribution to the data.

In [ ]:
# estimate 2log(BF)
null_bics = []
null_size = []
for i in range(51):
    x = power2[:,107:118+i].flatten() - power1[:,107:118+i].flatten()    # change sample size for each loop
    null_size.append(len(x))
    null_bics.append(bayes_factor_approx(x, null_pdf, 'norm'))

In [ ]:
# perform KS goodness-of-fit test
null_norm_dn = []
null_dn = []
null_ks_cv = []

for i in range(51):
    data = power2[:,107:118+i].flatten() - power1[:,107:118+i].flatten()
    npt = norm.fit(data)
    null_norm_dn.append(stats.kstest(data, lambda x: gaus_cdf(x, npt[0], npt[1], 
                                                           min(data), max(data)))[0])

    y, x = np.histogram(data, bins='auto', density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0
    popt = curve_fit(null_pdf, x, y, p0=np.std(data))[0]
    null_dn.append(stats.kstest(data, lambda x: null_cdf(x, popt, min(data), max(data)))[0])   
    
    null_ks_cv.append(1.36/np.sqrt(len(data)))

## 5. Plotting KS test stats and 2log(*BF*) against data size

In [ ]:
plt.figure(figsize=(13.5, 7))
plt.subplots_adjust(hspace=.4, wspace=.3)

# Bayesian model selection for comparison between CNN and N
plt.subplot(221)
plt.plot([size[0], size[-1]], [10, 10], 'k--', label="very strong\nevidence\nagainst $\mathcal{N}$")
plt.plot(size, bics, 'g.', label="twice the log\nof Bayes factors")
plt.xlabel("sample size", fontsize=14)
plt.ylabel("2S", fontsize=14)
plt.title("Bayesian model comparison between $\mathcal{N}$ and $\mathcal{CNN}$", fontsize=14)
plt.legend(loc='best', fontsize=12)
plt.grid()

# Bayesian model selection for comparison between delta CNN and N
plt.subplot(222)
plt.plot([size[0], size[-1]], [10, 10], 'k--', label="very strong\nevidence\nagainst $\mathcal{N}$")
plt.plot(null_size, null_bics, 'g.', label="twice the log\nof Bayes factors")
plt.xlabel("sample size", fontsize=14)
plt.ylabel("2S", fontsize=14)
plt.title("Bayesian model comparison between $\mathcal{N}$ and $\mathcal{\Delta CNN}$", fontsize=14)
plt.legend(loc='best', fontsize=12)
plt.grid()

# KS goodness-of-fit test for comparison between CNN and N
plt.subplot(223)
plt.plot(size, ks_cv, 'k--', label="critical value")
plt.plot(size, norm_dn, '.', label="$\mathcal{N}$ fit")
plt.plot(size, real_dn, '.', label="$\mathcal{CNN}$ fit")
plt.xlabel("sample size\n\n(a) fit to high-delay power", fontsize=14)
plt.ylabel("D", fontsize=14)
plt.title("KS goodness of fit tests for $\mathcal{N}$ and $\mathcal{CNN}$", fontsize=14)
plt.legend(loc='best', fontsize=12)
plt.grid()

# KS goodness-of-fit test for comparison between delta CNN and N
plt.subplot(224)
plt.plot(null_size, null_ks_cv, 'k--', label="critical value")
plt.plot(null_size, null_norm_dn, '.', label="$\mathcal{N}$ fit")
plt.plot(null_size, null_dn, '.', label="$\Delta\mathcal{CNN}$ fit")
plt.xlabel("sample size\n\n(b) fit to differences between high-delay power", fontsize=14)
plt.ylabel("D", fontsize=14)
plt.title("KS goodness of fit tests for $\mathcal{N}$ and $\Delta\mathcal{CNN}$", fontsize=14)
plt.legend(loc='best', fontsize=12)
plt.grid()

plt.show()